<a href="https://colab.research.google.com/github/roccaab/WaveletGAN/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Parametri di training
num_epochs = 100
batch_size = 32
latent_dim = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Inizializzazione del generatore e dei discriminatori
generator = Generator(latent_dim=latent_dim).to(device)
discriminators = [DiscriminatorWavelet().to(device) for _ in range(6)]

optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
# Ottimizzatore per i discriminatori: aggrega i parametri di tutti
optimizer_D = optim.Adam([p for disc in discriminators for p in disc.parameters()], lr=0.0002, betas=(0.5, 0.999))
criterion = nn.BCELoss()

# Simulazione di un dataloader (da sostituire con il proprio)
# Supponiamo di avere un dataset di accelerogrammi di 2000 campioni già segmentati attorno al picco PGA
# e un tensore di condizioni (magnitudo, PGA)
# Ad esempio:
dummy_signals = torch.randn(500, 2000)  # 500 segnali reali
dummy_conditions = torch.randn(500, 2)   # condizioni casuali

# Creiamo un semplice DataLoader
from torch.utils.data import TensorDataset, DataLoader
dataset = TensorDataset(dummy_signals, dummy_conditions)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Training Loop
for epoch in range(num_epochs):
    for real_signals, conditions in data_loader:
        real_signals = real_signals.to(device)
        conditions = conditions.to(device)

        # Normalizzazione del segnale
        real_signals = normalize_signal(real_signals)

        # Generazione di segnali sintetici
        z = torch.randn(real_signals.size(0), latent_dim, device=device)
        fake_signals = generator(z, conditions)
        fake_signals = normalize_signal(fake_signals)

        # Decomposizione wavelet per segnali reali e generati
        wavelet_real = wavelet_decomposition(real_signals)  # (batch, 6, 2000)
        wavelet_fake = wavelet_decomposition(fake_signals)

        # Aggiornamento dei Discriminatori
        optimizer_D.zero_grad()
        loss_D, loss_G = total_loss(discriminators, wavelet_real, wavelet_fake, conditions, criterion)
        loss_D.backward(retain_graph=True)
        optimizer_D.step()

        # Aggiornamento del Generatore
        optimizer_G.zero_grad()
        # Rigeneriamo il segnale sintetico per il passo del generatore
        z = torch.randn(real_signals.size(0), latent_dim, device=device)
        fake_signals = generator(z, conditions)
        fake_signals = normalize_signal(fake_signals)
        wavelet_fake = wavelet_decomposition(fake_signals)
        _, loss_G = total_loss(discriminators, wavelet_real, wavelet_fake, conditions, criterion)
        loss_G.backward()
        optimizer_G.step()

    print(f"Epoch {epoch+1}/{num_epochs}: Loss D = {loss_D.item():.4f}, Loss G = {loss_G.item():.4f}")

# Fase di inferenza: generazione di un accelerogramma sintetico
def generate_accelerogram(json_path, generator, latent_dim=100):
    """
    Legge il file JSON delle condizioni, genera un accelerogramma sintetico di 10 secondi
    e restituisce il segnale (normalizzato) e le sue componenti wavelet.
    """
    condition = textToVect(json_path).to(device)  # (1, 2)
    z = torch.randn(1, latent_dim, device=device)
    with torch.no_grad():
        fake_signal = generator(z, condition)  # (1, 2000)
        fake_signal = normalize_signal(fake_signal)
        wavelet_components = wavelet_decomposition(fake_signal)  # (1, 6, 2000)
    return fake_signal.cpu().numpy(), wavelet_components.cpu().numpy()

# Esempio di inferenza:
# Supponendo di avere un file "condition.json" con le chiavi "magnitudo" e "PGA"
# fake_accel, fake_wavelets = generate_accelerogram("condition.json", generator)
# plt.plot(fake_accel[0])
# plt.title("Accelerogramma Sintetico Generato")
# plt.show()
